In [ ]:
import math
import pandas as pd

In [ ]:
def twoDecimalPoint(data):
  data = float(format(data, '.2f'))

  return data

In [ ]:
si = pd.read_csv('/content/dataset_SI19.csv')
si.head()

,NIM,IPK
0,192410101001,3.623
1,192410101002,3.515
2,192410101003,3.846
3,192410101004,3.469
4,192410101005,3.031


In [ ]:
ti = pd.read_csv('/content/dataset_TI19.csv')
ti.head()

,NIM,IPK
0,192410102001,3.469
1,192410102002,3.838
2,192410102003,3.477
3,192410102004,2.778
4,192410102006,3.575


# SI 19


In [ ]:
si_min = twoDecimalPoint(si.IPK.min())
si_max = twoDecimalPoint(si.IPK.max())
si_range = twoDecimalPoint(si_max - si_min)
si_k = math.ceil(twoDecimalPoint(1+(3.33*math.log(len(si.IPK), 10))))
si_p = twoDecimalPoint(si_range/si_k)

def show(label, value):
  print(label, '=', value)

show('Min', si_min)
show('Max', si_max)
show('Range', si_range)
show('K', si_k)
show('P', si_p)

Min = 1.55
Max = 3.97
Range = 2.42
K = 9
P = 0.27


In [ ]:
def getInterval(Min, K, P):
  interval = []
  for i in range(K):
    if i == 0:
      interval.append([Min, Min+twoDecimalPoint(P-0.01)])
    else:
      a = twoDecimalPoint(interval[i-1][0]+P)
      b = twoDecimalPoint(a+twoDecimalPoint(P-0.01))
      interval.append([a, b])

  interval = pd.DataFrame(interval)
  interval.columns = ['batas_bawah', 'batas_atas']

  return interval

si_interval = getInterval(si_min, si_k, si_p)
si_interval

,batas_bawah,batas_atas
0,1.55,1.81
1,1.82,2.08
2,2.09,2.35
3,2.36,2.62
4,2.63,2.89
5,2.90,3.16
6,3.17,3.43
7,3.44,3.70
8,3.71,3.97


In [ ]:
def getFrekuensi(df, interval):
  frekuensi = []
  for i in range(len(interval)):
    counter = 0
    for j in df.IPK:
      if interval.loc[i].batas_bawah <= twoDecimalPoint(j) <= interval.loc[i].batas_atas:
        counter += 1

    frekuensi.append(counter)
  frekuensi = pd.DataFrame(frekuensi)
  frekuensi.columns = ['frekuensi']

  return pd.concat([interval, frekuensi], axis='columns')

si_interval = getFrekuensi(si, si_interval)
si_interval

,batas_bawah,batas_atas,frekuensi
0,1.55,1.81,1
1,1.82,2.08,0
2,2.09,2.35,0
3,2.36,2.62,2
4,2.63,2.89,5
5,2.90,3.16,4
6,3.17,3.43,35
7,3.44,3.70,66
8,3.71,3.97,27


In [ ]:
# menghitung jumlah frekuensi IPK prodi SI
jumlah_fi_si = si_interval.frekuensi.sum()
jumlah_fi_si

140

### Mean

In [ ]:
def getNilaiTengah(df):
  xi = []
  for i in range(len(df)):
    xi.append(twoDecimalPoint((df.loc[i].batas_bawah+df.loc[i].batas_atas)/2))
  
  xi = pd.DataFrame(xi)
  xi.columns = ['nilai_tengah']

  return pd.concat([df, xi], axis='columns')

si_interval = getNilaiTengah(si_interval)

In [ ]:
def getFixi(df):
  fixi = []
  for i in range(len(df)):
    fixi.append(
        twoDecimalPoint(df.loc[i].frekuensi*df.loc[i].nilai_tengah)
    )
  fixi = pd.DataFrame(fixi)
  fixi.columns = ['fi*xi']

  return pd.concat([df, fixi], axis='columns')

si_interval = getFixi(si_interval)

si_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi
0,1.55,1.81,1,1.68,1.68
1,1.82,2.08,0,1.95,0.00
2,2.09,2.35,0,2.22,0.00
3,2.36,2.62,2,2.49,4.98
4,2.63,2.89,5,2.76,13.80
5,2.90,3.16,4,3.03,12.12
6,3.17,3.43,35,3.30,115.50
7,3.44,3.70,66,3.57,235.62
8,3.71,3.97,27,3.84,103.68


In [ ]:
jumlah_fixi_si = si_interval['fi*xi'].sum()

mean = jumlah_fixi_si/jumlah_fi_si

print('mean =>', mean)

mean => 3.4812857142857148


### Median

In [ ]:
def getFK(df):
  fk = []
  for i in range(len(df)):
    if i == 0:
      fk.append(df.loc[i].frekuensi)
    else:
      fk.append(fk[i-1]+df.loc[i].frekuensi)
  fk = pd.DataFrame(fk)
  fk.columns = ['frekuensi_kumulatif']

  return pd.concat([df, fk], axis='columns')

si_interval = getFK(si_interval)
si_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif
0,1.55,1.81,1,1.68,1.68,1.0
1,1.82,2.08,0,1.95,0.00,1.0
2,2.09,2.35,0,2.22,0.00,1.0
3,2.36,2.62,2,2.49,4.98,3.0
4,2.63,2.89,5,2.76,13.80,8.0
5,2.90,3.16,4,3.03,12.12,12.0
6,3.17,3.43,35,3.30,115.50,47.0
7,3.44,3.70,66,3.57,235.62,113.0
8,3.71,3.97,27,3.84,103.68,140.0


In [ ]:
def getMedian(df, P):
  global si_range
  letak_median = twoDecimalPoint(df.frekuensi.sum()/2)
  for i in range(len(df)):
    if letak_median < df.loc[i].frekuensi_kumulatif:
      lm = twoDecimalPoint(df.loc[i].batas_bawah-0.5)
      cf = df.loc[i-1].frekuensi_kumulatif
      fm = df.loc[i].frekuensi
      w = P

      break

  return twoDecimalPoint(lm+w*(jumlah_fi_si/2-cf)/fm)

median = getMedian(si_interval, si_range)
print('Median =>', median)

Median => 3.78


### Modus

In [ ]:
def getModus(df, P):
  i = df.frekuensi.idxmax() # mencari index frekuensi terbanyak
  b = twoDecimalPoint(df.loc[i].batas_bawah-0.5)
  w = P
  f1 = df.loc[i-1].frekuensi
  f2 = df.loc[i+1].frekuensi

  return twoDecimalPoint(b+w*(f1/(f1+f2)))

modus = getModus(si_interval, si_range)
print('Modus =>',modus)

Modus => 4.31


### Range

In [ ]:
def getRange(df):
  batas_bawah = twoDecimalPoint(df.loc[0].batas_bawah-0.5)
  batas_atas = twoDecimalPoint(df.loc[len(df)-1].batas_atas+0.5)

  return batas_atas-batas_bawah

rentang = getRange(si_interval)

print('Range =>', rentang)

Range => 3.42


### Simpangan rata-rata

In [ ]:
average = twoDecimalPoint(jumlah_fixi_si/jumlah_fi_si)

In [ ]:
def getXiXbar(df):
  xixbar = []
  for i in df.nilai_tengah:
    xixbar.append(twoDecimalPoint(abs(i-average)))

  xixbar = pd.DataFrame(xixbar)
  xixbar.columns = ['xi-xbar']

  return pd.concat([df, xixbar], axis='columns')

si_interval = getXiXbar(si_interval)
si_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar
0,1.55,1.81,1,1.68,1.68,1.0,1.80
1,1.82,2.08,0,1.95,0.00,1.0,1.53
2,2.09,2.35,0,2.22,0.00,1.0,1.26
3,2.36,2.62,2,2.49,4.98,3.0,0.99
4,2.63,2.89,5,2.76,13.80,8.0,0.72
5,2.90,3.16,4,3.03,12.12,12.0,0.45
6,3.17,3.43,35,3.30,115.50,47.0,0.18
7,3.44,3.70,66,3.57,235.62,113.0,0.09
8,3.71,3.97,27,3.84,103.68,140.0,0.36


In [ ]:
def getFiXiXbar(df):
  fixixbar = []
  for i in range(len(df)):
    fixixbar.append(twoDecimalPoint(df.loc[i].frekuensi*df.loc[i]['xi-xbar']))
  
  fixixbar = pd.DataFrame(fixixbar)
  fixixbar.columns = ['fi[xi-xbar]']

  return pd.concat([df, fixixbar], axis='columns')

si_interval = getFiXiXbar(si_interval)
si_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar,fi[xi-xbar]
0,1.55,1.81,1,1.68,1.68,1.0,1.80,1.80
1,1.82,2.08,0,1.95,0.00,1.0,1.53,0.00
2,2.09,2.35,0,2.22,0.00,1.0,1.26,0.00
3,2.36,2.62,2,2.49,4.98,3.0,0.99,1.98
4,2.63,2.89,5,2.76,13.80,8.0,0.72,3.60
5,2.90,3.16,4,3.03,12.12,12.0,0.45,1.80
6,3.17,3.43,35,3.30,115.50,47.0,0.18,6.30
7,3.44,3.70,66,3.57,235.62,113.0,0.09,5.94
8,3.71,3.97,27,3.84,103.68,140.0,0.36,9.72


In [ ]:
jumlah_fixixbar_si = si_interval['fi[xi-xbar]'].sum()
jumlah_fixixbar_si

31.14

In [ ]:
meanDeviasi = twoDecimalPoint(jumlah_fixixbar_si/jumlah_fi_si)
print('Simpangan rata-rata =>', meanDeviasi)

Simpangan rata-rata => 0.22


### Standar Deviasi

In [ ]:
def getXiKuadrat(df):
  xikuadrat = []
  for i in df['nilai_tengah']:
    xikuadrat.append(twoDecimalPoint(i**2))

  xikuadrat = pd.DataFrame(xikuadrat)
  xikuadrat.columns = ['xi_kuadrat']

  return pd.concat([df, xikuadrat], axis='columns')

def getFiXiKuadrat(df):
  fixikuadrat = []
  for i in df['fi*xi']:
    fixikuadrat.append(twoDecimalPoint(i**2))

  fixikuadrat = pd.DataFrame(fixikuadrat)
  fixikuadrat.columns = ['fixi_kuadrat']

  return pd.concat([df, fixikuadrat], axis='columns')

si_interval = getXiKuadrat(si_interval)
si_interval = getFiXiKuadrat(si_interval)
si_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar,fi[xi-xbar],xi_kuadrat,fixi_kuadrat
0,1.55,1.81,1,1.68,1.68,1.0,1.80,1.80,2.82,2.82
1,1.82,2.08,0,1.95,0.00,1.0,1.53,0.00,3.80,0.00
2,2.09,2.35,0,2.22,0.00,1.0,1.26,0.00,4.93,0.00
3,2.36,2.62,2,2.49,4.98,3.0,0.99,1.98,6.20,24.80
4,2.63,2.89,5,2.76,13.80,8.0,0.72,3.60,7.62,190.44
5,2.90,3.16,4,3.03,12.12,12.0,0.45,1.80,9.18,146.89
6,3.17,3.43,35,3.30,115.50,47.0,0.18,6.30,10.89,13340.25
7,3.44,3.70,66,3.57,235.62,113.0,0.09,5.94,12.74,55516.78
8,3.71,3.97,27,3.84,103.68,140.0,0.36,9.72,14.75,10749.54


In [ ]:
varian = twoDecimalPoint((si_interval['fixi_kuadrat'].sum()-(si_interval['fi*xi'].sum()**2/jumlah_fi_si))/(jumlah_fi_si-1))
varian

563.13

In [ ]:
standar_deviasi = twoDecimalPoint(math.sqrt(varian))
print('Standar deviasi =>', standar_deviasi)

Standar deviasi => 23.73


# TI 19

In [ ]:
ti_min = twoDecimalPoint(ti.IPK.min())
ti_max = twoDecimalPoint(ti.IPK.max())
ti_range = twoDecimalPoint(ti_max - ti_min)
ti_k = math.ceil(twoDecimalPoint(1+(3.33*math.log(len(ti.IPK), 10))))
ti_p = twoDecimalPoint(ti_range/ti_k)

In [ ]:
show('Min', ti_min)
show('Max', ti_max)
show('Range', ti_range)
show('K', ti_k)
show('P', ti_p)

Min = 0.53
Max = 3.88
Range = 3.35
K = 8
P = 0.42


In [ ]:
ti_interval = getInterval(ti_min, ti_k, ti_p,)
ti_interval

,batas_bawah,batas_atas
0,0.53,0.94
1,0.95,1.36
2,1.37,1.78
3,1.79,2.20
4,2.21,2.62
5,2.63,3.04
6,3.05,3.46
7,3.47,3.88


In [ ]:
ti_interval = getFrekuensi(ti, ti_interval)
ti_interval

,batas_bawah,batas_atas,frekuensi
0,0.53,0.94,1
1,0.95,1.36,2
2,1.37,1.78,0
3,1.79,2.20,1
4,2.21,2.62,7
5,2.63,3.04,8
6,3.05,3.46,41
7,3.47,3.88,27


In [ ]:
# menghitung jumlah frekuensi IPK prodi SI
jumlah_fi_ti = ti_interval.frekuensi.sum()
jumlah_fi_ti

87

### Mean

In [ ]:
ti_interval = getNilaiTengah(ti_interval)

In [ ]:
ti_interval = getFixi(ti_interval)

ti_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi
0,0.53,0.94,1,0.73,0.73
1,0.95,1.36,2,1.16,2.32
2,1.37,1.78,0,1.58,0.00
3,1.79,2.20,1,2.00,2.00
4,2.21,2.62,7,2.42,16.94
5,2.63,3.04,8,2.83,22.64
6,3.05,3.46,41,3.25,133.25
7,3.47,3.88,27,3.67,99.09


In [ ]:
jumlah_fixi_ti = ti_interval['fi*xi'].sum()

mean = twoDecimalPoint(jumlah_fixi_ti/jumlah_fi_ti)

print('mean =>', mean)

mean => 3.18


### Median

In [ ]:
ti_interval = getFK(ti_interval)
ti_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif
0,0.53,0.94,1,0.73,0.73,1.0
1,0.95,1.36,2,1.16,2.32,3.0
2,1.37,1.78,0,1.58,0.00,3.0
3,1.79,2.20,1,2.00,2.00,4.0
4,2.21,2.62,7,2.42,16.94,11.0
5,2.63,3.04,8,2.83,22.64,19.0
6,3.05,3.46,41,3.25,133.25,60.0
7,3.47,3.88,27,3.67,99.09,87.0


In [ ]:
median = getMedian(ti_interval, ti_range)
print('Median =>', median)

Median => 6.72


### Modus

In [ ]:
modus = getModus(ti_interval, ti_range)
print('Modus =>',modus)

Modus => 3.32


### Range

In [ ]:
rentang = getRange(ti_interval)

print('Range =>', rentang)

Range => 4.35


### Simpangan rata-rata

In [ ]:
average = twoDecimalPoint(jumlah_fixi_si/jumlah_fi_si)

In [ ]:
ti_interval = getXiXbar(ti_interval)
ti_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar
0,0.53,0.94,1,0.73,0.73,1.0,2.75
1,0.95,1.36,2,1.16,2.32,3.0,2.32
2,1.37,1.78,0,1.58,0.00,3.0,1.90
3,1.79,2.20,1,2.00,2.00,4.0,1.48
4,2.21,2.62,7,2.42,16.94,11.0,1.06
5,2.63,3.04,8,2.83,22.64,19.0,0.65
6,3.05,3.46,41,3.25,133.25,60.0,0.23
7,3.47,3.88,27,3.67,99.09,87.0,0.19


In [ ]:
ti_interval = getFiXiXbar(ti_interval)
ti_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar,fi[xi-xbar]
0,0.53,0.94,1,0.73,0.73,1.0,2.75,2.75
1,0.95,1.36,2,1.16,2.32,3.0,2.32,4.64
2,1.37,1.78,0,1.58,0.00,3.0,1.90,0.00
3,1.79,2.20,1,2.00,2.00,4.0,1.48,1.48
4,2.21,2.62,7,2.42,16.94,11.0,1.06,7.42
5,2.63,3.04,8,2.83,22.64,19.0,0.65,5.20
6,3.05,3.46,41,3.25,133.25,60.0,0.23,9.43
7,3.47,3.88,27,3.67,99.09,87.0,0.19,5.13


In [ ]:
jumlah_fixixbar_ti = ti_interval['fi[xi-xbar]'].sum()
jumlah_fixixbar_ti

36.05

In [ ]:
meanDeviasi = twoDecimalPoint(jumlah_fixixbar_ti/jumlah_fi_ti)
print('Simpangan rata-rata =>', meanDeviasi)

Simpangan rata-rata => 0.41


### Standar Deviasi

In [ ]:
ti_interval = getXiKuadrat(ti_interval)
ti_interval = getFiXiKuadrat(ti_interval)
ti_interval

,batas_bawah,batas_atas,frekuensi,nilai_tengah,fi*xi,frekuensi_kumulatif,xi-xbar,fi[xi-xbar],xi_kuadrat,fixi_kuadrat
0,0.53,0.94,1,0.73,0.73,1.0,2.75,2.75,0.53,0.53
1,0.95,1.36,2,1.16,2.32,3.0,2.32,4.64,1.35,5.38
2,1.37,1.78,0,1.58,0.00,3.0,1.90,0.00,2.50,0.00
3,1.79,2.20,1,2.00,2.00,4.0,1.48,1.48,4.00,4.00
4,2.21,2.62,7,2.42,16.94,11.0,1.06,7.42,5.86,286.96
5,2.63,3.04,8,2.83,22.64,19.0,0.65,5.20,8.01,512.57
6,3.05,3.46,41,3.25,133.25,60.0,0.23,9.43,10.56,17755.56
7,3.47,3.88,27,3.67,99.09,87.0,0.19,5.13,13.47,9818.83


In [ ]:
varian = twoDecimalPoint((ti_interval['fixi_kuadrat'].sum()-(ti_interval['fi*xi'].sum()**2/jumlah_fi_ti))/(jumlah_fi_ti-1))
varian

319.79

In [ ]:
standar_deviasi = twoDecimalPoint(math.sqrt(varian))
print('Standar deviasi =>', standar_deviasi)

Standar deviasi => 17.88
